# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 10:22AM,254148,10,8868345-BO,Eywa Pharma Inc.,Released
1,Jan 3 2023 10:22AM,254148,10,8838521-2,Eywa Pharma Inc.,Released
2,Jan 3 2023 10:22AM,254148,10,8837460-2-BO,Eywa Pharma Inc.,Released
3,Jan 3 2023 10:22AM,254148,10,8925635-BO,Eywa Pharma Inc.,Released
4,Jan 3 2023 10:22AM,254148,10,8926705-BO,Eywa Pharma Inc.,Released
5,Jan 3 2023 10:22AM,254148,10,8926710-BO,Eywa Pharma Inc.,Released
6,Jan 3 2023 10:22AM,254148,10,8925636-BO,Eywa Pharma Inc.,Released
7,Jan 3 2023 10:22AM,254148,10,8954900,Eywa Pharma Inc.,Released
8,Jan 3 2023 10:22AM,254148,10,8954896-BO,Eywa Pharma Inc.,Released
9,Jan 3 2023 10:22AM,254148,10,8955463-BO,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,254142,Released,1
33,254144,Released,1
34,254147,Released,1
35,254148,Released,18
36,254149,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254142,NaN,1.0
254144,NaN,1.0
254147,NaN,1.0
254148,NaN,18.0
254149,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254090,46.0,16.0
254104,0.0,1.0
254107,1.0,0.0
254108,0.0,24.0
254111,0.0,53.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254090,46,16
254104,0,1
254107,1,0
254108,0,24
254111,0,53


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254090,46,16
1,254104,0,1
2,254107,1,0
3,254108,0,24
4,254111,0,53


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254090,46,16
1,254104,,1
2,254107,1,
3,254108,,24
4,254111,,53


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 10:22AM,254148,10,Eywa Pharma Inc.
18,Jan 3 2023 10:21AM,254149,20,"HVL, LLC dba Atrium Innovations"
19,Jan 3 2023 10:18AM,254139,10,ISDIN Corporation
42,Jan 3 2023 10:17AM,254138,10,ISDIN Corporation
79,Jan 3 2023 10:17AM,254137,10,ISDIN Corporation
113,Jan 3 2023 10:17AM,254136,10,ISDIN Corporation
158,Jan 3 2023 10:16AM,254147,10,"Methapharm, Inc."
159,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc."
160,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc."
161,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 3 2023 10:22AM,254148,10,Eywa Pharma Inc.,,18
1,Jan 3 2023 10:21AM,254149,20,"HVL, LLC dba Atrium Innovations",,1
2,Jan 3 2023 10:18AM,254139,10,ISDIN Corporation,,23
3,Jan 3 2023 10:17AM,254138,10,ISDIN Corporation,,37
4,Jan 3 2023 10:17AM,254137,10,ISDIN Corporation,,34
5,Jan 3 2023 10:17AM,254136,10,ISDIN Corporation,,45
6,Jan 3 2023 10:16AM,254147,10,"Methapharm, Inc.",,1
7,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",,1
8,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",,1
9,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:22AM,254148,10,Eywa Pharma Inc.,18,
1,Jan 3 2023 10:21AM,254149,20,"HVL, LLC dba Atrium Innovations",1,
2,Jan 3 2023 10:18AM,254139,10,ISDIN Corporation,23,
3,Jan 3 2023 10:17AM,254138,10,ISDIN Corporation,37,
4,Jan 3 2023 10:17AM,254137,10,ISDIN Corporation,34,
5,Jan 3 2023 10:17AM,254136,10,ISDIN Corporation,45,
6,Jan 3 2023 10:16AM,254147,10,"Methapharm, Inc.",1,
7,Jan 3 2023 10:06AM,254144,21,"NBTY Global, Inc.",1,
8,Jan 3 2023 10:05AM,254142,21,"NBTY Global, Inc.",1,
9,Jan 3 2023 10:01AM,254140,19,"AdvaGen Pharma, Ltd",3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:22AM,254148,10,Eywa Pharma Inc.,18,
1,Jan 3 2023 10:21AM,254149,20,"HVL, LLC dba Atrium Innovations",1,
2,Jan 3 2023 10:18AM,254139,10,ISDIN Corporation,23,
3,Jan 3 2023 10:17AM,254138,10,ISDIN Corporation,37,
4,Jan 3 2023 10:17AM,254137,10,ISDIN Corporation,34,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:22AM,254148,10,Eywa Pharma Inc.,18.0,NaN
1,Jan 3 2023 10:21AM,254149,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
2,Jan 3 2023 10:18AM,254139,10,ISDIN Corporation,23.0,NaN
3,Jan 3 2023 10:17AM,254138,10,ISDIN Corporation,37.0,NaN
4,Jan 3 2023 10:17AM,254137,10,ISDIN Corporation,34.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 10:22AM,254148,10,Eywa Pharma Inc.,18.0,0.0
1,Jan 3 2023 10:21AM,254149,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
2,Jan 3 2023 10:18AM,254139,10,ISDIN Corporation,23.0,0.0
3,Jan 3 2023 10:17AM,254138,10,ISDIN Corporation,37.0,0.0
4,Jan 3 2023 10:17AM,254137,10,ISDIN Corporation,34.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5336687,635.0,0.0
12,762362,3.0,0.0
15,762381,59.0,22.0
19,1016469,20.0,47.0
20,762382,3.0,0.0
21,508286,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5336687,635.0,0.0
1,12,762362,3.0,0.0
2,15,762381,59.0,22.0
3,19,1016469,20.0,47.0
4,20,762382,3.0,0.0
5,21,508286,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,635.0,0.0
1,12,3.0,0.0
2,15,59.0,22.0
3,19,20.0,47.0
4,20,3.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,635.0
1,12,Released,3.0
2,15,Released,59.0
3,19,Released,20.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,0.0,0.0,22.0,47.0,0.0,0.0
Released,635.0,3.0,59.0,20.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,0.0,0.0,22.0,47.0,0.0,0.0
1,Released,635.0,3.0,59.0,20.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,0.0,0.0,22.0,47.0,0.0,0.0
1,Released,635.0,3.0,59.0,20.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()